# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 20 2022 2:34PM,253522,10,MSP216942,"Methapharm, Inc.",Released
1,Dec 20 2022 2:34PM,253522,10,MSP216940,"Methapharm, Inc.",Released
2,Dec 20 2022 2:34PM,253521,10,0086262272,ISDIN Corporation,Released
3,Dec 20 2022 2:31PM,253520,10,Eme-106716,Emerginnova,Released
4,Dec 20 2022 2:31PM,253520,10,Enova-11124,Emerginnova,Released
5,Dec 20 2022 2:18PM,253519,10,Yusen-8927359,Yusen – 3D Matrix,Released
6,Dec 20 2022 1:57PM,253517,10,8926753,MedStone Pharma LLC,Released
7,Dec 20 2022 1:57PM,253517,10,8926754,MedStone Pharma LLC,Released
8,Dec 20 2022 1:57PM,253517,10,8926755,MedStone Pharma LLC,Released
9,Dec 20 2022 1:57PM,253517,10,8926756,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,253517,Released,11
26,253519,Released,1
27,253520,Released,2
28,253521,Released,1
29,253522,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253517,NaN,NaN,11.0
253519,NaN,NaN,1.0
253520,NaN,NaN,2.0
253521,NaN,NaN,1.0
253522,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253419,0.0,0.0,1.0
253421,0.0,0.0,1.0
253446,13.0,0.0,2.0
253452,10.0,19.0,19.0
253455,0.0,4.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253419,0,0,1
253421,0,0,1
253446,13,0,2
253452,10,19,19
253455,0,4,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253419,0,0,1
1,253421,0,0,1
2,253446,13,0,2
3,253452,10,19,19
4,253455,0,4,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253419,,,1
1,253421,,,1
2,253446,13,,2
3,253452,10,19,19
4,253455,,4,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 20 2022 2:34PM,253522,10,"Methapharm, Inc."
2,Dec 20 2022 2:34PM,253521,10,ISDIN Corporation
3,Dec 20 2022 2:31PM,253520,10,Emerginnova
5,Dec 20 2022 2:18PM,253519,10,Yusen – 3D Matrix
6,Dec 20 2022 1:57PM,253517,10,MedStone Pharma LLC
17,Dec 20 2022 1:15PM,253512,15,Lonza
18,Dec 20 2022 1:14PM,253511,12,Hush Hush
29,Dec 20 2022 12:59PM,253509,10,Bio-PRF
30,Dec 20 2022 12:31PM,253505,10,ISDIN Corporation
33,Dec 20 2022 12:30PM,253503,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 20 2022 2:34PM,253522,10,"Methapharm, Inc.",,,2
1,Dec 20 2022 2:34PM,253521,10,ISDIN Corporation,,,1
2,Dec 20 2022 2:31PM,253520,10,Emerginnova,,,2
3,Dec 20 2022 2:18PM,253519,10,Yusen – 3D Matrix,,,1
4,Dec 20 2022 1:57PM,253517,10,MedStone Pharma LLC,,,11
5,Dec 20 2022 1:15PM,253512,15,Lonza,,,1
6,Dec 20 2022 1:14PM,253511,12,Hush Hush,,11,
7,Dec 20 2022 12:59PM,253509,10,Bio-PRF,,,1
8,Dec 20 2022 12:31PM,253505,10,ISDIN Corporation,,,3
9,Dec 20 2022 12:30PM,253503,10,ISDIN Corporation,,,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 20 2022 2:34PM,253522,10,"Methapharm, Inc.",2,,
1,Dec 20 2022 2:34PM,253521,10,ISDIN Corporation,1,,
2,Dec 20 2022 2:31PM,253520,10,Emerginnova,2,,
3,Dec 20 2022 2:18PM,253519,10,Yusen – 3D Matrix,1,,
4,Dec 20 2022 1:57PM,253517,10,MedStone Pharma LLC,11,,
5,Dec 20 2022 1:15PM,253512,15,Lonza,1,,
6,Dec 20 2022 1:14PM,253511,12,Hush Hush,,11,
7,Dec 20 2022 12:59PM,253509,10,Bio-PRF,1,,
8,Dec 20 2022 12:31PM,253505,10,ISDIN Corporation,3,,
9,Dec 20 2022 12:30PM,253503,10,ISDIN Corporation,16,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 20 2022 2:34PM,253522,10,"Methapharm, Inc.",2,,
1,Dec 20 2022 2:34PM,253521,10,ISDIN Corporation,1,,
2,Dec 20 2022 2:31PM,253520,10,Emerginnova,2,,
3,Dec 20 2022 2:18PM,253519,10,Yusen – 3D Matrix,1,,
4,Dec 20 2022 1:57PM,253517,10,MedStone Pharma LLC,11,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 20 2022 2:34PM,253522,10,"Methapharm, Inc.",2.0,NaN,NaN
1,Dec 20 2022 2:34PM,253521,10,ISDIN Corporation,1.0,NaN,NaN
2,Dec 20 2022 2:31PM,253520,10,Emerginnova,2.0,NaN,NaN
3,Dec 20 2022 2:18PM,253519,10,Yusen – 3D Matrix,1.0,NaN,NaN
4,Dec 20 2022 1:57PM,253517,10,MedStone Pharma LLC,11.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 20 2022 2:34PM,253522,10,"Methapharm, Inc.",2.0,0.0,0.0
1,Dec 20 2022 2:34PM,253521,10,ISDIN Corporation,1.0,0.0,0.0
2,Dec 20 2022 2:31PM,253520,10,Emerginnova,2.0,0.0,0.0
3,Dec 20 2022 2:18PM,253519,10,Yusen – 3D Matrix,1.0,0.0,0.0
4,Dec 20 2022 1:57PM,253517,10,MedStone Pharma LLC,11.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4563042,90.0,4.0,0.0
12,253511,0.0,11.0,0.0
15,1267366,65.0,0.0,13.0
18,253471,1.0,0.0,0.0
19,506873,20.0,19.0,10.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4563042,90.0,4.0,0.0
1,12,253511,0.0,11.0,0.0
2,15,1267366,65.0,0.0,13.0
3,18,253471,1.0,0.0,0.0
4,19,506873,20.0,19.0,10.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,90.0,4.0,0.0
1,12,0.0,11.0,0.0
2,15,65.0,0.0,13.0
3,18,1.0,0.0,0.0
4,19,20.0,19.0,10.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,90.0
1,12,Released,0.0
2,15,Released,65.0
3,18,Released,1.0
4,19,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19
Status,,,,,
Completed,0.0,0.0,13.0,0.0,10.0
Executing,4.0,11.0,0.0,0.0,19.0
Released,90.0,0.0,65.0,1.0,20.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19
0,Completed,0.0,0.0,13.0,0.0,10.0
1,Executing,4.0,11.0,0.0,0.0,19.0
2,Released,90.0,0.0,65.0,1.0,20.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19
0,Completed,0.0,0.0,13.0,0.0,10.0
1,Executing,4.0,11.0,0.0,0.0,19.0
2,Released,90.0,0.0,65.0,1.0,20.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()